In [10]:
import nbformat

# Read your notebook (assuming version 4 for example purposes)
nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

%run ml_project_normalized.ipynb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned color
cleaned breed
dropped columns


/tmp/ipykernel_898512/1199118304.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_series = pd.to_datetime(df['intake_time'], errors='coerce')


cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned color
cleaned breed
Done running ml_project.ipynb.


In [11]:
# from sklearn.decomposition import PCA

# # Apply PCA to retain 95% of variance
# pca = PCA(n_components=0.95)
# X_train = pca.fit_transform(X_train)
# X_test  = pca.transform(X_test)

# print(f"[INFO] Reduced dimensions with PCA: {X_train.shape[1]} components retained.")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.metrics import make_scorer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
import pandas as pd

def train_RF_classifier(X_train, y_train, X_test, cat_cols, target_samples_per_class=3000):
  """
  Trains an Extremely Randomized Trees model using class weighting and RandomizedSearchCV.

  Parameters:
    X_train (pd.DataFrame or np.ndarray): Fully preprocessed training features.
    y_train (array-like): Encoded training labels.
    X_test (pd.DataFrame or np.ndarray): Fully preprocessed test features.
    rare_classes (list): List of int-encoded rare class labels.
    target_samples_per_class (int): Target number of samples per rare class (currently unused).

  Returns:
    best_estimator: Trained model.
    test_predictions: Predictions on the test set.
  """

  '''
    # Make copies to avoid modifying original data
  X_train = X_train.copy()
  X_test = X_test.copy()

  # Drop hour_sin and hour_cos if they exist
  drop_cols = ['hour_sin', 'hour_cos']
  existing_cols = [col for col in drop_cols if col in X_train.columns]
  '''
  all_cols = X_train.columns.tolist()

  # Convert column names to positional indices
  cat_indices = [all_cols.index(col) for col in cat_cols]



  # Compute class weights for imbalanced training set
  class_labels = np.unique(y_train)
  class_weights = compute_class_weight('balanced', classes=class_labels, y=y_train)
  class_weight_dict = dict(zip(class_labels, class_weights))
  print("Class weights:", class_weight_dict)

  categorical_transformer = Pipeline(steps=[
    #   ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values (if any)
      ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-Hot Encode categorical features
  ])

    # Combine both transformations into a single ColumnTransformer
  preprocessor = ColumnTransformer(
      transformers=[
          ('cat', categorical_transformer, cat_indices)
      ])

  # Create a pipeline that first transforms the data and then applies Logistic Regression
  pipeline = Pipeline(steps=[
    #  ('smote', SMOTENC(categorical_features=cat_cols, random_state=42, sampling_strategy=sampling_strategy)),
      ('preprocessor', preprocessor),
      ('rf', RandomForestClassifier(class_weight=class_weight_dict, random_state=42, n_jobs=-1))  # You can adjust max_iter as needed
  ])


  '''
  # Build model pipeline (only model step, encoding was done externally)
  model = RandomForestClassifier(
    n_estimators=100,
    class_weight=class_weight_dict,
    random_state=42,
    n_jobs=-1
  )
  '''
  # Define hyperparameter search space
  param_dist = {
    'rf__n_estimators': [100, 200, 300],
    'rf__max_depth': [10, 20, None], 
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4]
  }
  

  search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=5,
    cv=skf,
    scoring='balanced_accuracy',
    verbose=2,
    random_state=42,
    n_jobs=2
  )

  print(f"\n[INFO] Starting training with {len(X_train)} samples")
  search.fit(X_train, y_train)
  print("[INFO] Training complete.")

  # Plotting feature importance
  import matplotlib.pyplot as plt
  
  '''
  feature_names = X_train.columns  # assumes X_train is still a DataFrame
  importances   = search.best_estimator_.feature_importances_
  indices       = np.argsort(importances)[::-1]

  print("\\n[INFO] Feature ranking:")
  for rank, idx in enumerate(indices[:20], 1):
    print(f"{rank}. {feature_names[idx]} — importance: {importances[idx]:.4f}")

  plt.figure(figsize=(12, 6))
  plt.title("Top 20 Feature Importances")
  plt.bar(range(20), importances[indices[:20]], align="center")
  plt.xticks(range(20), [feature_names[i] for i in indices[:20]], rotation=45, ha='right')
  plt.tight_layout()
  plt.show()
  '''

  print("Best parameters:", search.best_params_)
  print("Best CV accuracy:", search.best_score_)

  # Optionally evaluate again with cross_val_score if needed
  try:
    cv_scores = cross_val_score(
      search.best_estimator_,
      X_train,
      y_train,
      cv=skf,
      scoring='balanced_accuracy',
      verbose=3
    )
    print("Generalization accuracy (via cross_val_score):", cv_scores.mean())
  except Exception as e:
    print(f"Cross-validation scoring failed: {e}")

  # Final test prediction
  test_predictions = search.best_estimator_.predict(X_test)

  return search.best_estimator_, test_predictions


In [ ]:
# Encode the target variable.
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
print('Encoding mapping:', le.classes_)

best_model, test_predictions = train_Bclassifier(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    cat_cols=cat_cols,
    num_cols=num_cols

)

# Convert predictions back to original labels.
classification_report_with_accuracy_score(y_test, test_predictions)
#print(importance)

Encoding mapping: ['Adoption' 'Died' 'Euthanasia' 'Return to Owner' 'Transfer']
Rare classes:
  2: Euthanasia
  1: Died


NameError: name 'cat_cols' is not defined